In [1]:
import pandas as pd

try:
    # Đọc file CSV, chỉ đọc 10000 dòng đầu
    df = pd.read_csv('data_llm.csv', nrows=10000)

    # Lưu DataFrame chứa 10000 dòng đầu vào một file CSV mới
    df.to_csv('data_1000.csv', index=False, encoding='utf-8')

    print("Đã lấy 10000 dòng đầu của 'data.csv' và lưu vào file 'data_10000.csv'")

except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'data.csv'. Vui lòng đảm bảo file tồn tại trong thư mục hiện tại.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

Đã lấy 10000 dòng đầu của 'data.csv' và lưu vào file 'data_10000.csv'


In [5]:
import pandas as pd
from datasets import Dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
import torch


In [6]:

# 1. Đọc dữ liệu
df = pd.read_csv("data_1000.csv")  # Thay 'path_to_your_file.csv' bằng đường dẫn đến file của bạn


In [7]:

# Chuyển dữ liệu thành Dataset của Hugging Face
dataset = Dataset.from_pandas(df)

In [8]:
# 2. Tải mô hình và tokenizer
model_name = "facebook/bart-large-cnn"  # Hoặc "t5-large" nếu bạn muốn dùng T5
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [9]:

# 3. Tokenize dữ liệu
def preprocess_function(examples):
    inputs = examples['Contents']
    targets = examples['Summary']
    
    # Tokenize inputs và outputs
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Áp dụng hàm preprocess cho dữ liệu
dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
from transformers import Trainer, TrainingArguments
import torch

# Bắt model và tokenizer chạy trên CPU
model = model.to("cpu")

training_args = TrainingArguments(
    output_dir="./results",               # Thư mục lưu kết quả
    evaluation_strategy="epoch",          # Đánh giá sau mỗi epoch
    learning_rate=2e-5,                   # Tốc độ học
    per_device_train_batch_size=1,        # ↓ Giảm batch size để tránh quá tải RAM/VRAM
    per_device_eval_batch_size=1,         # ↓ Tương tự với eval
    num_train_epochs=3,                   # Số epoch huấn luyện
    weight_decay=0.01,                    # Lệch trọng số
    save_total_limit=1,                   # Chỉ giữ lại 1 checkpoint để tiết kiệm dung lượng
    fp16=False,                           # ❌ Tắt fp16 vì không dùng GPU
    gradient_checkpointing=False,         # ❌ Tắt vì không cần thiết với batch nhỏ & CPU
    dataloader_num_workers=0,             # Worker = 0 để tránh lỗi treo hoặc OOM trên Colab/máy yếu
    logging_steps=10,                     # Ghi log sau mỗi 10 step
    logging_dir="./logs",                 # Lưu log tensorboard
)


In [19]:

# 5. Khởi tạo Trainer
trainer = Trainer(
    model=model,                          # Mô hình
    args=training_args,                   # Các tham số huấn luyện
    train_dataset=dataset,                # Dữ liệu huấn luyện
    eval_dataset=dataset,                 # Dữ liệu kiểm tra
)


In [20]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [21]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.45 GiB is allocated by PyTorch, and 19.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)